<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">

*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

<!--NAVIGATION-->
< [Introducing Pandas Objects](03.01-Introducing-Pandas-Objects.ipynb) | [Contents](Index.ipynb) | [Operating on Data in Pandas](03.03-Operations-in-Pandas.ipynb) >

<a href="https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.02-Data-Indexing-and-Selection.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>


# Data Indexing and Selection

In [Chapter 2](02.00-Introduction-to-NumPy.ipynb), we looked in detail at methods and tools to access, set, and modify values in NumPy arrays.
These included indexing (e.g., ``arr[2, 1]``), slicing (e.g., ``arr[:, 1:5]``), masking (e.g., ``arr[arr > 0]``), fancy indexing (e.g., ``arr[0, [1, 5]]``), and combinations thereof (e.g., ``arr[:, [1, 5]]``).
Here we'll look at similar means of accessing and modifying values in Pandas ``Series`` and ``DataFrame`` objects.
If you have used the NumPy patterns, the corresponding patterns in Pandas will feel very familiar, though there are a few quirks to be aware of.

We'll start with the simple case of the one-dimensional ``Series`` object, and then move on to the more complicated two-dimesnional ``DataFrame`` object.

## Data Selection in Series

As we saw in the previous section, a ``Series`` object acts in many ways like a one-dimensional NumPy array, and in many ways like a standard Python dictionary.
If we keep these two overlapping analogies in mind, it will help us to understand the patterns of data indexing and selection in these arrays.

### Series as dictionary

Like a dictionary, the ``Series`` object provides a mapping from a collection of keys to a collection of values:

In [2]:
import pandas as pd
import numpy as np

In [20]:
my_series = pd.Series(
    [1, 2, 3, 4],
    index=["a", "b", "c", "d"],
    name="My Values",
)
my_series

a    1
b    2
c    3
d    4
Name: My Values, dtype: int64

In [21]:
my_series.values

array([1, 2, 3, 4])

In [22]:
my_series["a"]

1

In [25]:
subset = my_series["a":"c"]
subset

a    1
b    2
c    3
Name: My Values, dtype: int64

In [29]:
my_series["b"] = -99

In [30]:
subset

a    99
b   -99
c     3
Name: My Values, dtype: int64

In [26]:
subset["a"] = 99
subset

a    99
b     2
c     3
Name: My Values, dtype: int64

In [28]:
my_series

a    99
b     2
c     3
d     4
Name: My Values, dtype: int64

In [24]:
my_series[["a", "c"]]

a    1
c    3
Name: My Values, dtype: int64

We can also use dictionary-like Python expressions and methods to examine the keys/indices and values:

``Series`` objects can even be modified with a dictionary-like syntax.
Just as you can extend a dictionary by assigning to a new key, you can extend a ``Series`` by assigning to a new index value:

This easy mutability of the objects is a convenient feature: under the hood, Pandas is making decisions about memory layout and data copying that might need to take place; the user generally does not need to worry about these issues.

### Series as one-dimensional array

A ``Series`` builds on this dictionary-like interface and provides array-style item selection via the same basic mechanisms as NumPy arrays – that is, *slices*, *masking*, and *fancy indexing*.
Examples of these are as follows:

In [ ]:
# slicing by explicit index

In [ ]:
# slicing by implicit integer index

In [ ]:
# masking

In [ ]:
# fancy indexing

Among these, slicing may be the source of the most confusion.
Notice that when slicing with an explicit index (i.e., ``data['a':'c']``), the final index is *included* in the slice, while when slicing with an implicit index (i.e., ``data[0:2]``), the final index is *excluded* from the slice.

### Indexers: loc, iloc, and ix

These slicing and indexing conventions can be a source of confusion.
For example, if your ``Series`` has an explicit integer index, an indexing operation such as ``data[1]`` will use the explicit indices, while a slicing operation like ``data[1:3]`` will use the implicit Python-style index.

In [ ]:
# explicit index when indexing

In [ ]:
# implicit index when slicing

Because of this potential confusion in the case of integer indexes, Pandas provides some special *indexer* attributes that explicitly expose certain indexing schemes.
These are not functional methods, but attributes that expose a particular slicing interface to the data in the ``Series``.

First, the ``loc`` attribute allows indexing and slicing that always references the explicit index:

The ``iloc`` attribute allows indexing and slicing that always references the implicit Python-style index:

A third indexing attribute, ``ix``, is a hybrid of the two, and for ``Series`` objects is equivalent to standard ``[]``-based indexing.
The purpose of the ``ix`` indexer will become more apparent in the context of ``DataFrame`` objects, which we will discuss in a moment.

One guiding principle of Python code is that "explicit is better than implicit."
The explicit nature of ``loc`` and ``iloc`` make them very useful in maintaining clean and readable code; especially in the case of integer indexes, I recommend using these both to make code easier to read and understand, and to prevent subtle bugs due to the mixed indexing/slicing convention.

## Data Selection in DataFrame

Recall that a ``DataFrame`` acts in many ways like a two-dimensional or structured array, and in other ways like a dictionary of ``Series`` structures sharing the same index.
These analogies can be helpful to keep in mind as we explore data selection within this structure.

In [47]:
my_df = pd.DataFrame({
    "column A": [1, 2, 3],
    "column B": ["a", "b", "c"]
    },
    index=[2000, 2001, 2002]
)
my_df

,column A,column B
2000,1,a
2001,2,b
2002,3,c


In [42]:
import os

In [43]:
DATA_DIRECTORY = "data/"

In [45]:
cal_cities_csv = os.path.join(DATA_DIRECTORY, "california_cities.csv")
cal_cities_csv

'data/california_cities.csv'

In [65]:
cal_cities = pd.read_csv(
    cal_cities_csv,
    index_col="city"
).drop(columns=["Unnamed: 0"])
cal_cities.head()

,latd,longd,elevation_m,elevation_ft,population_total,area_total_sq_mi,area_land_sq_mi,area_water_sq_mi,area_total_km2,area_land_km2,area_water_km2,area_water_percent
city,,,,,,,,,,,,
Adelanto,34.576111,-117.432778,875.0,2871.0,31765,56.027,56.009,0.018,145.107,145.062,0.046,0.03
AgouraHills,34.153333,-118.761667,281.0,922.0,20330,7.822,7.793,0.029,20.260,20.184,0.076,0.37
Alameda,37.756111,-122.274444,NaN,33.0,75467,22.960,10.611,12.349,59.465,27.482,31.983,53.79
Albany,37.886944,-122.297778,NaN,43.0,18969,5.465,1.788,3.677,14.155,4.632,9.524,67.28
Alhambra,34.081944,-118.135000,150.0,492.0,83089,7.632,7.631,0.001,19.766,19.763,0.003,0.01


In [87]:
my_series = pd.Series(
    [1, 2, 3, 4], 
    index=    [1, 2, 3, 4]
)
my_series

1    1
2    2
3    3
4    4
dtype: int64

In [89]:
my_series.loc[1]

1

In [90]:
my_series.iloc[1]

2

In [93]:
cal_cities.iloc[:5, cal_cities.columns.get_loc("latd")]

city
Adelanto       34.576111
AgouraHills    34.153333
Alameda        37.756111
Albany         37.886944
Alhambra       34.081944
Name: latd, dtype: float64

In [97]:
cal_cities.loc["Adelanto":"Albany", cal_cities.columns[-1]]

city
Adelanto        0.03
AgouraHills     0.37
Alameda        53.79
Albany         67.28
Name: area_water_percent, dtype: float64

In [103]:
# cal_cities.iloc[:, 2:4]

In [114]:
cal_cities.loc["Adelanto":"Albany"]

,latd,longd,elevation_m,elevation_ft,population_total,area_total_sq_mi,area_land_sq_mi,area_water_sq_mi,area_total_km2,area_land_km2,area_water_km2,area_water_percent
city,,,,,,,,,,,,
Adelanto,34.576111,-117.432778,875.0,2871.0,31765,56.027,56.009,0.018,145.107,145.062,0.046,0.03
AgouraHills,34.153333,-118.761667,281.0,922.0,20330,7.822,7.793,0.029,20.260,20.184,0.076,0.37
Alameda,37.756111,-122.274444,NaN,33.0,75467,22.960,10.611,12.349,59.465,27.482,31.983,53.79
Albany,37.886944,-122.297778,NaN,43.0,18969,5.465,1.788,3.677,14.155,4.632,9.524,67.28


In [115]:
cal_cities.iloc[0:3]

,latd,longd,elevation_m,elevation_ft,population_total,area_total_sq_mi,area_land_sq_mi,area_water_sq_mi,area_total_km2,area_land_km2,area_water_km2,area_water_percent
city,,,,,,,,,,,,
Adelanto,34.576111,-117.432778,875.0,2871.0,31765,56.027,56.009,0.018,145.107,145.062,0.046,0.03
AgouraHills,34.153333,-118.761667,281.0,922.0,20330,7.822,7.793,0.029,20.260,20.184,0.076,0.37
Alameda,37.756111,-122.274444,NaN,33.0,75467,22.960,10.611,12.349,59.465,27.482,31.983,53.79


In [139]:
# Sort the index to do partial indexing
cal_cities = cal_cities.sort_index()

In [137]:
cal_cities.index.is_monotonic_increasing

True

In [140]:
"Alb" < "Albany"

True

In [141]:
"Albz" > "Albany"

True

In [138]:
cal_cities.loc["Adelanto":"Alb"]

,latd,longd,elevation_m,elevation_ft,population_total,area_total_sq_mi,area_land_sq_mi,area_water_sq_mi,area_total_km2,area_land_km2,area_water_km2,area_water_percent
city,,,,,,,,,,,,
Adelanto,34.576111,-117.432778,875.0,2871.0,31765,56.027,56.009,0.018,145.107,145.062,0.046,0.03
AgouraHills,34.153333,-118.761667,281.0,922.0,20330,7.822,7.793,0.029,20.260,20.184,0.076,0.37
Alameda,37.756111,-122.274444,NaN,33.0,75467,22.960,10.611,12.349,59.465,27.482,31.983,53.79


In [120]:
cal_cities.loc[
    (50 < cal_cities["area_water_percent"]
    & (cal_cities["area_water_percent"] < 70),
    "latd":"elevation_m"
]

,latd,longd,elevation_m
city,,,
Alameda,37.756111,-122.274444,NaN
Albany,37.886944,-122.297778,NaN
Hercules,38.017222,-122.288611,24.0
NewportBeach,33.616667,-117.897500,3.0
Pinole,38.004444,-122.291111,31.0
SantaBarbara,34.425833,-119.714167,15.0
SouthSanFrancisco,37.656111,-122.425556,4.0
Tiburon,37.873611,-122.456667,4.0


In [104]:
cal_cities.loc[
    ~(cal_cities["area_water_percent"] > 50),
    ["latd", "longd", "area_water_percent"]
]

,latd,longd,area_water_percent
city,,,
Adelanto,34.576111,-117.432778,0.03
AgouraHills,34.153333,-118.761667,0.37
Alhambra,34.081944,-118.135000,0.01
AlisoViejo,33.575000,-117.725556,0.00
Alturas,41.487222,-120.542500,0.57
AmadorCity,38.419444,-120.824167,0.00
AmericanCanyon,38.168056,-122.252500,0.17
Anaheim,33.836111,-117.889722,1.92
Anderson,40.452222,-122.296667,3.74


In [78]:
cal_cities.loc["Adelanto":"Albany", "latd":"elevation_m"]

,latd,longd,elevation_m
city,,,
Adelanto,34.576111,-117.432778,875.0
AgouraHills,34.153333,-118.761667,281.0
Alameda,37.756111,-122.274444,NaN
Albany,37.886944,-122.297778,NaN


In [72]:
cal_cities["elevation_m"]

city
Adelanto            875.0
AgouraHills         281.0
Alameda               NaN
Albany                NaN
Alhambra            150.0
AlisoViejo          127.0
Alturas            1332.0
AmadorCity          280.0
AmericanCanyon       14.0
Anaheim              48.0
Anderson            132.0
AngelsCamp          420.0
Antioch              13.0
AppleValley         898.0
Arcadia             147.0
Arcata                NaN
ArroyoGrande         36.0
Artesia              16.0
Arvin               137.0
Atascadero          268.0
Atherton             18.0
Atwater              46.0
Auburn              374.0
Avalon                9.0
Avenal              246.0
Azusa               186.0
Bakersfield           NaN
BaldwinPark         114.0
Banning             716.0
Barstow             664.0
                    ...  
Vista                99.0
Walnut              171.0
WalnutCreek           NaN
Wasco               100.0
Waterford            52.0
Watsonville           9.0
Weed               1044.0
WestCov

### DataFrame as a dictionary

The first analogy we will consider is the ``DataFrame`` as a dictionary of related ``Series`` objects.
Let's return to our example of areas and populations of states:

The individual ``Series`` that make up the columns of the ``DataFrame`` can be accessed via dictionary-style indexing of the column name:

Equivalently, we can use attribute-style access with column names that are strings:

This attribute-style column access actually accesses the exact same object as the dictionary-style access:

Though this is a useful shorthand, keep in mind that it does not work for all cases!
For example, if the column names are not strings, or if the column names conflict with methods of the ``DataFrame``, this attribute-style access is not possible.
For example, the ``DataFrame`` has a ``pop()`` method, so ``data.pop`` will point to this rather than the ``"pop"`` column:

In particular, you should avoid the temptation to try column assignment via attribute (i.e., use ``data['pop'] = z`` rather than ``data.pop = z``).

Like with the ``Series`` objects discussed earlier, this dictionary-style syntax can also be used to modify the object, in this case adding a new column:

This shows a preview of the straightforward syntax of element-by-element arithmetic between ``Series`` objects; we'll dig into this further in [Operating on Data in Pandas](03.03-Operations-in-Pandas.ipynb).

### DataFrame as two-dimensional array

As mentioned previously, we can also view the ``DataFrame`` as an enhanced two-dimensional array.
We can examine the raw underlying data array using the ``values`` attribute:

With this picture in mind, many familiar array-like observations can be done on the ``DataFrame`` itself.
For example, we can transpose the full ``DataFrame`` to swap rows and columns:

When it comes to indexing of ``DataFrame`` objects, however, it is clear that the dictionary-style indexing of columns precludes our ability to simply treat it as a NumPy array.
In particular, passing a single index to an array accesses a row:

and passing a single "index" to a ``DataFrame`` accesses a column:

Thus for array-style indexing, we need another convention.
Here Pandas again uses the ``loc``, ``iloc``, and ``ix`` indexers mentioned earlier.
Using the ``iloc`` indexer, we can index the underlying array as if it is a simple NumPy array (using the implicit Python-style index), but the ``DataFrame`` index and column labels are maintained in the result:

Similarly, using the ``loc`` indexer we can index the underlying data in an array-like style but using the explicit index and column names:

The ``ix`` indexer allows a hybrid of these two approaches:

Keep in mind that for integer indices, the ``ix`` indexer is subject to the same potential sources of confusion as discussed for integer-indexed ``Series`` objects.

Any of the familiar NumPy-style data access patterns can be used within these indexers.
For example, in the ``loc`` indexer we can combine masking and fancy indexing as in the following:

Any of these indexing conventions may also be used to set or modify values; this is done in the standard way that you might be accustomed to from working with NumPy:

To build up your fluency in Pandas data manipulation, I suggest spending some time with a simple ``DataFrame`` and exploring the types of indexing, slicing, masking, and fancy indexing that are allowed by these various indexing approaches.

### Additional indexing conventions

There are a couple extra indexing conventions that might seem at odds with the preceding discussion, but nevertheless can be very useful in practice.
First, while *indexing* refers to columns, *slicing* refers to rows:

Such slices can also refer to rows by number rather than by index:

Similarly, direct masking operations are also interpreted row-wise rather than column-wise:

These two conventions are syntactically similar to those on a NumPy array, and while these may not precisely fit the mold of the Pandas conventions, they are nevertheless quite useful in practice.

<!--NAVIGATION-->
< [Introducing Pandas Objects](03.01-Introducing-Pandas-Objects.ipynb) | [Contents](Index.ipynb) | [Operating on Data in Pandas](03.03-Operations-in-Pandas.ipynb) >

<a href="https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.02-Data-Indexing-and-Selection.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>
